In [4]:
# import packages
import arcpy
import pandas as pd
from pathlib import Path
import arcgis

# Set environment settings
arcpy.env.overwriteOutput = True
# network path to connection files
filePath = "F:\\GIS\\DB_CONNECT"
# database file path 
sdeBase = Path(filePath) / "Vector.sde"

# environment settings
arcpy.env.workspace = "memory"
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(26910)
sr = arcpy.SpatialReference(26910)

In [ ]:
districts_db  = Path(sdeBase) / "SDE.Planning/SDE.District"
try:
    with arcpy.EnvManager(outputZFlag="Disabled"):
        arcpy.conversion.FeatureClassToGeodatabase(
            Input_Features=str(districts_db),
            Output_Geodatabase=r"C:\GIS\Scratch.gdb"
        )
    sdf_district_noz = pd.DataFrame.spatial.from_featureclass('C:/GIS/Scratch.gdb/District')
    sdf_district_noz.spatial.sr = sr
    print("Feature class successfully converted and loaded into DataFrame.")
except arcpy.ExecuteError as e:
    print(f"Error in executing tool: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# 
years = [2018, 2019, 2020, 2021, 2022, 2023]
# paths to feature classes
parcel_db     = Path(sdeBase) / "SDE.Parcels\SDE.Parcel_History_Attributed"
try:
    with arcpy.EnvManager(outputZFlag="Disabled"):
        arcpy.conversion.FeatureClassToGeodatabase(
            Input_Features=str(parcel_db),
            Output_Geodatabase=r"C:\GIS\Scratch.gdb"
        )
except arcpy.ExecuteError as e:
    print(f"Error in executing tool: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# parcel layer
# paths to feature classes
parcel_db     = 'C:/GIS/Scratch.gdb/Parcel_History_Attributed'
# make feature layer from scratch gdb of parcels
arcpy.management.MakeFeatureLayer(
    in_features=parcel_db,
    out_layer="Parcels",
    where_clause='YEAR >= 2018 AND YEAR <= 2023'
)
arcpy.management.AlterField(
    in_table="Parcels",
    field="Residential_Units",
    new_field_name="RES",
    new_field_alias="RES"
)
arcpy.management.AlterField(
    in_table="Parcels",
    field="TouristAccommodation_Units",
    new_field_name="TAU",
    new_field_alias="TAU"
) 
arcpy.management.AlterField(
    in_table="Parcels",
    field="CommercialFloorArea_SqFt",
    new_field_name="CFA",
    new_field_alias="CFA"
)

In [ ]:
import arcpy

# List of years to process
years = [2018, 2019, 2020, 2021, 2022, 2023]

# Paths to feature classes
districts_fc = 'C:/GIS/Scratch.gdb/District'
parcels_fc = 'C:/GIS/Scratch.gdb/Parcel_History_Attributed'
output_directory = r"C:\GIS\Scratch.gdb"  # Path to output directory for spatial join results

# Define the fields to be summed up
fields_to_sum = ['RES', 'TAU', 'CFA']
# Fields to keep from Districts feature class
fields_to_keep = ['ZONING_ID', 'ZONING_DESCRIPTION', 'PLAN_ID', 'PLAN_NAME', 'PLAN_TYPE']

# Loop over each year
for year in years:
    print(f"Processing year: {year}")
    
    # Create a where clause to filter parcels by the year
    where_clause = f"YEAR = {year}"
    
    # Create a temporary feature class for the parcels filtered by the year
    year_parcels_fc = f"memory\\parcels_{year}"
    
    # Make a feature layer for the parcels filtered by year
    arcpy.management.MakeFeatureLayer(parcels_fc, "parcels_layer", where_clause)
    arcpy.management.CopyFeatures("parcels_layer", year_parcels_fc)
    # how many parcels have RES, TAU, and CFA > 0 and null ZONING_ID?
    # select parcels with RES, TAU, and CFA > 0 and null ZONING_ID
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=year_parcels_fc,
        selection_type="NEW_SELECTION",
        where_clause="RES > 0 AND TAU > 0 AND CFA > 0 AND ZONING_ID IS NULL"
    )
    # get count of selected parcels
    count = arcpy.management.GetCount(year_parcels_fc)
    print(f"Number of parcels with RES, TAU, and CFA > 0 and null ZONING_ID: {count}")
    # total of RES
    total_res = arcpy.da.TableToNumPyArray(year_parcels_fc, "RES", skip_nulls=True)
    total_res = total_res['RES'].sum()
    print(f"Total number of residential units: {total_res}")
    # total of TAU
    total_tau = arcpy.da.TableToNumPyArray(year_parcels_fc, "TAU", skip_nulls=True)
    total_tau = total_tau['TAU'].sum()
    print(f"Total number of tourist accommodation units: {total_tau}")
    # total of CFA
    total_cfa = arcpy.da.TableToNumPyArray(year_parcels_fc, "CFA", skip_nulls=True)
    total_cfa = total_cfa['CFA'].sum()
    print(f"Total number of commercial floor area: {total_cfa}")


In [ ]:
# spatial join of parcels to districts to get ZONING_ID for parcels layer
# Define the output feature class for the spatial join
output_fc = f"{output_directory}\\parcels_{year}_joined"
# Perform the spatial join
arcpy.analysis.SpatialJoin(
    target_features=year_parcels_fc,
    join_features=districts_fc,
    out_feature_class=output_fc,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    match_option="INTERSECT",
    field_mapping="ZONING_ID \"ZONING_ID\" true true false 50 Text 0 0,First,#,District,ZONING_ID,-1,-1;ZONING_DESCRIPTION \"ZONING_DESCRIPTION\" true true false 50 Text 0 0,First,#,District,ZONING_DESCRIPTION,-1,-1;PLAN_ID \"PLAN_ID\" true true false 50 Text 0 0,First,#,District,PLAN_ID,-1,-1;PLAN_NAME \"PLAN_NAME\" true true false 50 Text 0 0,First,#,District,PLAN_NAME,-1,-1;PLAN_TYPE \"PLAN_TYPE\" true true false 50 Text 0 0,First,#,District,PLAN_TYPE,-1,-1",
    search_radius=None,
    distance_field_name=None
)
#

In [ ]:
import arcpy

# List of years to process
years = [2018, 2019, 2020, 2021, 2022, 2023]

# Paths to feature classes
districts_fc = 'C:/GIS/Scratch.gdb/District'
parcels_fc = 'C:/GIS/Scratch.gdb/Parcel_History_Attributed'
output_directory = r"C:\GIS\Scratch.gdb"  # Path to output directory for spatial join results

# Define the fields to be summed up
fields_to_sum = ['RES', 'TAU', 'CFA']
# Fields to keep from Districts feature class
fields_to_keep = ['ZONING_ID', 'ZONING_DESCRIPTION', 'PLAN_ID', 'PLAN_NAME', 'PLAN_TYPE']

# Loop over each year
for year in years:
    print(f"Processing year: {year}")
    
    # Create a where clause to filter parcels by the year
    where_clause = f"YEAR = {year}"
    
    # Create a temporary feature class for the parcels filtered by the year
    year_parcels_fc = f"memory\\parcels_{year}"
    
    # Make a feature layer for the parcels filtered by year
    arcpy.management.MakeFeatureLayer(parcels_fc, "parcels_layer", where_clause)
    arcpy.management.CopyFeatures("parcels_layer", year_parcels_fc)
    
    # Check feature counts
    districts_count = arcpy.management.GetCount(districts_fc)
    parcels_count = arcpy.management.GetCount(year_parcels_fc)
    print(f"Districts feature count: {districts_count}")
    print(f"Parcels feature count for year {year}: {parcels_count}")
    
    # Calculate total RES, TAU, CFA for the parcels for this year
    total_res_parcels, total_tau_parcels, total_cfa_parcels = 0, 0, 0
    with arcpy.da.SearchCursor(year_parcels_fc, fields_to_sum) as cursor:
        for row in cursor:
            total_res_parcels += row[0] if row[0] else 0
            total_tau_parcels += row[1] if row[1] else 0
            total_cfa_parcels += row[2] if row[2] else 0
    print(f"Total RES from parcels for year {year}: {total_res_parcels}")
    print(f"Total TAU from parcels for year {year}: {total_tau_parcels}")
    print(f"Total CFA from parcels for year {year}: {total_cfa_parcels}")
    
    # Set output filename for the current year
    output_fc = f"{output_directory}\\SpatialJoin_District_Parcel_{year}"

    # Initialize the FieldMappings object for the spatial join
    field_mappings = arcpy.FieldMappings()

    # Add FieldMappings for the fields to be summed (RES, TAU, CFA)
    for field in fields_to_sum:
        field_map_sum = arcpy.FieldMap()
        field_map_sum.addInputField(year_parcels_fc, field)
        field_map_sum.mergeRule = "SUM"
        field_mappings.addFieldMap(field_map_sum)

    # Add FieldMappings for the fields to keep (ZONING_ID, PLAN_ID, etc.)
    for field in fields_to_keep:
        field_map_text = arcpy.FieldMap()
        field_map_text.addInputField(districts_fc, field)
        field_map_text.mergeRule = "FIRST"
        field_mappings.addFieldMap(field_map_text)

    # Perform the spatial join for the current year with the "INTERSECT" match option
    arcpy.analysis.SpatialJoin(districts_fc, 
                               year_parcels_fc, 
                               output_fc, 
                               join_operation="JOIN_ONE_TO_ONE",
                               join_type="KEEP_ALL",
                               field_mapping=field_mappings,
                               match_option="CONTAINS")

    # Check output feature count after spatial join
    output_count = arcpy.management.GetCount(output_fc)
    print(f"Output feature count for year {year}: {output_count}")

    # Optionally, verify if the sums worked by querying the output FC
    total_res_output, total_tau_output, total_cfa_output = 0, 0, 0
    with arcpy.da.SearchCursor(output_fc, fields_to_sum) as cursor:
        for row in cursor:
            total_res_output += row[0] if row[0] else 0
            total_tau_output += row[1] if row[1] else 0
            total_cfa_output += row[2] if row[2] else 0
    
    # Compare the totals between parcels and output from spatial join
    print(f"Total RES from spatial join output for year {year}: {total_res_output}")
    print(f"Total TAU from spatial join output for year {year}: {total_tau_output}")
    print(f"Total CFA from spatial join output for year {year}: {total_cfa_output}")
    
    # Print a comparison of the totals
    print(f"Comparison for year {year}:")
    print(f"RES: Parcels = {total_res_parcels}, Output = {total_res_output}")
    print(f"TAU: Parcels = {total_tau_parcels}, Output = {total_tau_output}")
    print(f"CFA: Parcels = {total_cfa_parcels}, Output = {total_cfa_output}")

# Cleanup in-memory feature classes
arcpy.management.Delete("in_memory")
print("All spatial joins complete.")

In [ ]:
# get spatial join feature classes as data frams
years = [2018, 2019, 2020, 2021, 2022, 2023]
# paths to feature classes
for year in years:
    spjn_fc = f'C:/GIS/Scratch.gdb/SpatialJoin_District_Parcel_{year}'
    sdf = pd.DataFrame.spatial.from_featureclass(spjn_fc)
    sdf.spatial.sr = sr
    sdf['YEAR'] = year
    # append to new dataframe using concat for each year
    if year == 2018:
        df = sdf
    else:
        df = pd.concat([df, sdf], ignore_index=True)

df = df[['YEAR','ZONING_ID', 'ZONING_DESCRIPTION', 'PLAN_ID', 'PLAN_NAME', 'PLAN_TYPE', 'RES', 'TAU', 'CFA','SHAPE']]

df.info()
print(f"Feature class successfully loaded into DataFrame for year {year}.")

In [ ]:
# get sum of RES for each year
df.groupby('YEAR')['CFA'].sum()


In [ ]:
# pivot by year and zoning id
df_pivot = df.pivot_table(index=['ZONING_ID', 'ZONING_DESCRIPTION', 'PLAN_ID', 'PLAN_NAME', 'PLAN_TYPE'], columns='YEAR', values=['RES', 'TAU', 'CFA'], aggfunc='sum')
df_pivot

In [ ]:
# 
years = [2018, 2019, 2020, 2021, 2022, 2023]
# paths to feature classes
parcel_db     = 'C:/GIS/Scratch.gdb/Parcel_History_Attributed'
try:
    # summarize within for each year and filter Parcels by year
    for year in years: 
        # filter parcels by year
        parcel_year = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=parcel_db,
            selection_type="NEW_SELECTION",
            where_clause=f"YEAR = {year}"
        )
        # feature layer of districts
        arcpy.management.MakeFeatureLayer(
            in_features='C:/GIS/Scratch.gdb/District',
            out_layer="Districts"
        )
        arcpy.analysis.SummarizeWithin(
            in_polygons='Districts',
            in_sum_features=parcel_year,
            out_feature_class=rf"C:\GIS\Scratch.gdb\District_Dev_{str(year)}",
            keep_all_polygons="KEEP_ALL",
            sum_fields = [['RES', 'SUM'], ['TAU', 'SUM'], ['CFA', 'SUM']],
            # sum_shape="NO_SHAPE_SUM",
            # shape_unit="ACRES",
            # group_field=None,
            # add_min_maj="NO_MIN_MAJ",
            # add_group_percent="NO_PERCENT",
            # out_group_table=None
        )
        print(f"Summarize within for {year} completed.")
except arcpy.ExecuteError as e:
    print(f"Error in executing tool: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
arcpy.analysis.SummarizeWithin(
    in_polygons="C:/GIS/Scratch.gdb/District",
    in_sum_features="C:/GIS/Scratch.gdb/Parcel_History_Attributed",
    out_feature_class=r"C:\GIS\Scratch.gdb\District_Dev_2018_test",
    keep_all_polygons="KEEP_ALL",
    sum_fields="RES Sum;TAU Sum;CFA Sum",
    sum_shape="NO_SHAPE_SUM",
    shape_unit="ACRES",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)